In [2]:
import numpy as np
import random
import xarray as xr
import pandas as pd
import datetime as dt
import time
import math
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras import regularizers

import import_ipynb
import sys
import os 
import network_arch as network

importing Jupyter notebook from network_arch.ipynb


In [ ]:
#%% >>>>> ADDITIONAL FUNCTIONS >>>>>
def is_ndjf(month):
    return np.logical_or(month<=2, month>=11)

def is_ndjfm(month):
    return np.logical_or(month<=3, month>=11)

def is_mjja(month):
    return np.logical_and(month>=5, month<=8)

def is_amjja(month):
    return np.logical_and(month>=4, month<=8)

def is_mjjas(month):
    return np.logical_and(month>=5, month<=9)

def is_mam(month):
    return np.logical_and(month>=3, month<=5)

def is_may(month):
    return np.logical_and(month>=5, month<=5)

def is_aug(month):
    return np.logical_and(month>=8, month<=8)


#%% >>>>> NETWORK SETUP >>>>>
# MAKE THE NN ARCHITECTURE
def make_model():
    # Define and train the model
    tf.keras.backend.clear_session()
    model = network.defineNN(HIDDENS,
                             input1_shape = X_train.shape[1],
                             output_shape=NLABEL,
                             ridge_penalty1=RIDGE1,
                             dropout=DROPOUT,
                             act_fun='relu',
                             network_seed=NETWORK_SEED)
    
    loss_function = tf.keras.losses.CategoricalCrossentropy()    
    model.compile(
                  optimizer = tf.keras.optimizers.Adam(learning_rate=LR_INIT),
                  loss = loss_function,
                  metrics = [
                      tf.keras.metrics.CategoricalAccuracy(name="categorical_accuracy", dtype=None),
                      metrics.PredictionAccuracy(NLABEL)
                      ]
                  )           
    return model, loss_function

#---------------------------------------------------
#LEARNING RATE CALLBACK FUNCTION
def scheduler(epoch, lr):
    # This function keeps the initial learning rate for the first ten epochs
    # and decreases it exponentially after that.
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)
#---------------------------------------------------

def moving_average(a, n) :      #a is the input array, n is the window size
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

def rolling_average(data_array, window_size, dim):
    """
    Calculate the rolling average for an xarray data array.

    Parameters:
        data_array (xarray.DataArray): The input xarray data array.
        window_size (int): The size of the rolling window.
        dim (str or list): The dimension(s) along which to perform the rolling average.

    Returns:
        xarray.DataArray: The rolling average data array.
    """
    return data_array.rolling(**{dim: window_size}, center=True).mean()

In [1]:
from scipy.stats import poisson
import math 

def poisson_weight(value, lambd):
    # Calculate the Poisson probability mass function (PMF)
    weight = (math.exp(-lambd) * lambd**value) / math.factorial(value)
    return weight

def calculate_poisson_weights(lambd, max_value):
    weights = [poisson_weight(value, lambd) for value in range(max_value + 1)]
    return weights


In [ ]:
def centered_weighted_rolling_average(arr, weights):
    window_size = len(weights)
    
    # Pad the array to handle boundary cases
    left_pad = window_size // 2
    right_pad = window_size - left_pad - 1
    padded_arr = np.pad(arr, (left_pad, right_pad), mode='constant', constant_values=0)
    
    # Perform the convolution operation with fixed weights
    convolved = np.convolve(padded_arr, weights[::-1], mode='valid')
    
    # Compute the centered weighted rolling average
    result = convolved / np.sum(weights)
    
    return result


In [2]:
def forward_weighted_rolling_average(arr, weights):
    
    padded_arr = np.pad(arr, (0, len(weights) - 1), mode='constant', constant_values=0)
    
    # Perform the convolution operation with fixed weights
    convolved = np.convolve(padded_arr, weights[::-1], mode='valid')
    
    # Compute the centered weighted rolling average
    result = convolved / np.sum(weights)
    
    return result


In [ ]:
def backward_weighted_rolling_average(arr, weights):
    
    padded_arr = np.pad(arr, (0, len(weights) - 1), mode='constant', constant_values=0)
    
    # Perform the convolution operation with fixed weights
    convolved = np.convolve(padded_arr, weights[::-1], mode='valid')
    
    # Compute the centered weighted rolling average
    result = convolved / np.sum(weights)
    
    return result

In [4]:
def uncentered_weighted_rolling_average(arr, weights, center_element):
    window_size = len(weights)
    
    #Calculate the necessary padding
    left_pad = center_element
    right_pad = window_size - center_element - 1
    padded_arr = np.pad(arr, (left_pad, right_pad), mode='constant', constant_values=0)
    
    # Perform the convolution operation with fixed weights
    convolved = np.convolve(padded_arr, weights[::-1], mode='valid')
    
    # Compute the uncentered weighted rolling average
    result = convolved / np.sum(weights)
    
    return result

In [ ]:
def confusion_matrix(predclasses, targclasses):

    class_names = np.unique(targclasses)

    table = []
    for pred_class in class_names:
        row = []
        for true_class in class_names:
            row.append(100 * np.mean(predclasses[targclasses == true_class] == pred_class))
        table.append(row)
    class_titles_t = ["T(Light)", "T(Heavy)"]
    class_titles_p = ["P(Light)", "P(Heavy)"]
    conf_matrix = pd.DataFrame(table, index=class_titles_p, columns=class_titles_t)
    display(conf_matrix.style.background_gradient(cmap='Blues').format("{:.1f}"))

In [1]:
def find_indices_above_threshold(arr, threshold, axis):
    # Create a boolean mask of values above the threshold
    mask = arr > threshold

    # Use np.argwhere to find the indices where the mask is True
    indices = np.argwhere(mask)

    # Filter the indices based on the specified axis
    indices = indices[:, axis]

    return indices